# Crime Analysis (2015)

1. PySpark **environment setup**
2. Data source(mongodb, cassandra etc) and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get one or multiple **random samples** from the data set to better understand what the data is all about
4. Columns groups **basic profiling** to better understand our data set:
  1. **Timing related** columns basic profiling
  2. **Crime related** columns basic profiling
  3. **Issue related** columns basic profiling
5. **Answer some business questions** to improve service
  1. **Seasonality of crimes** 
  2. **Top Crimes/ Boroughs**



## 1. PySpark environment setup

In [184]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Data source and Spark data abstraction (DataFrame) setup

In [185]:
NYcrimesDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("NYPD_Complaint_Data_Historic_2015.csv")

## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame

In [186]:
from IPython.display import display, Markdown

NYcrimesDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % NYcrimesDF.count()))

root
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- CMPLNT_TO_DT: string (nullable = true)
 |-- CMPLNT_TO_TM: string (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- JURIS_DESC: string (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PREM_TYP_DESC: string (nullable = true)
 |-- PARKS_NM: string (nullable = true)
 |-- HADEVELOPT: string (nullable = true)
 |-- X_COORD_CD: integer (nullable = true)


This DataFrame has **468576 rows**.

### B. Get one or multiple random samples from the data set

In [187]:
NYcrimesDF.cache() # optimization to make the processing faster
NYcrimesDF.sample(False, 0.1).take(2)

[Row(CMPLNT_NUM=443856764, CMPLNT_FR_DT='1/1/15', month=1, day=1, year=2015, CMPLNT_FR_TM='8:00:00 AM', hour=8, CMPLNT_TO_DT='10/2/15', CMPLNT_TO_TM='9:55:00', RPT_DT='10/2/15', KY_CD=351, OFNS_DESC='CRIMINAL MISCHIEF & RELATED OF', PD_CD=258, PD_DESC='CRIMINAL MISCHIEF 4TH, GRAFFIT', CRM_ATPT_CPTD_CD='COMPLETED', LAW_CAT_CD='MISDEMEANOR', JURIS_DESC='N.Y. POLICE DEPT', BORO_NM='BROOKLYN', ADDR_PCT_CD=61, LOC_OF_OCCUR_DESC=None, PREM_TYP_DESC='OTHER', PARKS_NM=None, HADEVELOPT=None, X_COORD_CD=997856, Y_COORD_CD=156879, Latitude=40.59727062, Longitude=-73.95100754, Lat_Lon='(40.597270615, -73.951007536)'),
 Row(CMPLNT_NUM=451238388, CMPLNT_FR_DT='1/1/15', month=1, day=1, year=2015, CMPLNT_FR_TM='8:00:00 AM', hour=8, CMPLNT_TO_DT='9/17/15', CMPLNT_TO_TM='11:00:00', RPT_DT='9/17/15', KY_CD=351, OFNS_DESC='CRIMINAL MISCHIEF & RELATED OF', PD_CD=258, PD_DESC='CRIMINAL MISCHIEF 4TH, GRAFFIT', CRM_ATPT_CPTD_CD='COMPLETED', LAW_CAT_CD='MISDEMEANOR', JURIS_DESC='N.Y. POLICE DEPT', BORO_NM='BRO

## 4. Columns groups basic profiling to better understand our data set
### A. Timing related columns basic crime profiling

In [190]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import split,col
from pyspark.sql import Row

print ("Summary of columns OccurDate, OccurTime, endDate, endTime, reportDate:")
NYcrimesDF.select("CMPLNT_FR_DT","CMPLNT_FR_TM","CMPLNT_TO_DT","CMPLNT_TO_TM", "RPT_DT").summary().show()

print("Checking for nulls on columns OccurDate, OccurTime, endDate, endTime, reportDate:")
NYcrimesDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["CMPLNT_FR_DT","CMPLNT_FR_TM","CMPLNT_TO_DT","CMPLNT_TO_TM", "RPT_DT"]]).show()

print("Checking amount of distinct values in columns OccurDate, OccurTime, endDate, endTime, reportDate:")
NYcrimesDF.select([countDistinct(c).alias(c) for c in ["CMPLNT_FR_DT","CMPLNT_FR_TM","CMPLNT_TO_DT","CMPLNT_TO_TM", "RPT_DT"]]).show()

print ("Most and least frequent occurrences for OccurDate, OccurTime, OccurDay, and OccurMonth:")
OccurTimeDF = NYcrimesDF.groupBy("CMPLNT_FR_TM").agg(count(lit(1)).alias("Total"))
OccurDateDF = NYcrimesDF.groupBy("CMPLNT_FR_DT").agg(count(lit(1)).alias("Total"))
OccurdayDF  = NYcrimesDF.groupBy("day").agg(count(lit(1)).alias("Total"))
OccurmonthDF = NYcrimesDF.groupBy("month").agg(count(lit(1)).alias("Total"))

leastFreqOccurTime    = OccurTimeDF.orderBy(col("Total").asc()).first()
mostFreqOccurTime     = OccurTimeDF.orderBy(col("Total").desc()).first()
leastFreqOccurDate    = OccurDateDF.orderBy(col("Total").asc()).first()
mostFreqOccurDate     = OccurDateDF.orderBy(col("Total").desc()).first()
leastFreqOccurDay     = OccurdayDF.orderBy(col("Total").asc()).first()
mostFreqOccurDay      = OccurdayDF.orderBy(col("Total").desc()).first()
leastFreqOccurMonth   = OccurmonthDF.orderBy(col("Total").asc()).first()
mostFreqOccurMonth    = OccurmonthDF.orderBy(col("Total").desc()).first()



print("Most frequent OccurDate: %s (%d occurrences)" % (mostFreqOccurDate ["CMPLNT_FR_DT"], mostFreqOccurDate ["Total"]))
print("Least frequent OccurDate: %s (%d occurrences)" % (leastFreqOccurDate ["CMPLNT_FR_DT"], leastFreqOccurDate ["Total"]))

print("Most frequent OccurTime: %s (%d occurrences)" % (mostFreqOccurTime  ["CMPLNT_FR_TM"], mostFreqOccurTime  ["Total"]))
print("Least frequent OccurTime: %s (%d occurrences)" % (leastFreqOccurTime ["CMPLNT_FR_TM"], leastFreqOccurTime ["Total"]))

print("Most frequent OccurDay: %s (%d occurrences)" % (mostFreqOccurDay  ["day"], mostFreqOccurDay  ["Total"]))
print("Least frequent OccurDay: %s (%d occurrences)" % (leastFreqOccurDay ["day"], leastFreqOccurDay ["Total"]))

print("Most frequent OccurMonth: %s (%d occurrences)" % (mostFreqOccurMonth  ["month"], mostFreqOccurMonth  ["Total"]))
print("Least frequent OccurMonth: %s (%d occurrences)" % (leastFreqOccurMonth ["month"], leastFreqOccurMonth ["Total"]))



Summary of columns OccurDate, OccurTime, endDate, endTime, reportDate:
+-------+------------+------------+------------+------------+------+
|summary|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|RPT_DT|
+-------+------------+------------+------------+------------+------+
|  count|      468576|      468576|      378945|      379123|468576|
|   mean|        null|        null|        null|        null|  null|
| stddev|        null|        null|        null|        null|  null|
|    min|      1/1/15| 10:00:00 AM|      1/1/15|     0:00:00|1/1/15|
|    25%|        null|        null|        null|        null|  null|
|    50%|        null|        null|        null|        null|  null|
|    75%|        null|        null|        null|        null|  null|
|    max|      9/9/15|  9:59:00 PM|      9/9/15|     9:59:00|9/9/15|
+-------+------------+------------+------------+------------+------+

Checking for nulls on columns OccurDate, OccurTime, endDate, endTime, reportDate:
+------------+----

### B. Crime related columns basic profiling

In [13]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of columns Offense, LawCategory, Borough, PremisesDesc:")
NYcrimesDF.select("OFNS_DESC", "LAW_CAT_CD", "BORO_NM", "PREM_TYP_DESC").summary().show()

print("Checking for nulls on columns Offense, LawCategory, Borough, PremisesDesc:")
NYcrimesDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["OFNS_DESC", "LAW_CAT_CD", "BORO_NM", "PREM_TYP_DESC"]]).show()

print("Checking amount of distinct values in columns Offense, LawCategory, Borough, PremisesDesc:")
NYcrimesDF.select([countDistinct(c).alias(c) for c in ["OFNS_DESC", "LAW_CAT_CD", "BORO_NM", "PREM_TYP_DESC"]]).show()

print ("Most and least frequent occurrences for Offense, LawCategory, Borough, PremisesDesc columns:")
OffenseDF = NYcrimesDF.groupBy("OFNS_DESC").agg(count(lit(1)).alias("Total"))
LawCategoryDF   = NYcrimesDF.groupBy("LAW_CAT_CD").agg(count(lit(1)).alias("Total"))
BoroughDF    = NYcrimesDF.groupBy("BORO_NM").agg(count(lit(1)).alias("Total"))
PremisesDF      = NYcrimesDF.groupBy("PREM_TYP_DESC").agg(count(lit(1)).alias("Total"))

leastFreqOffense    = OffenseDF.orderBy(col("Total").asc()).first()
mostFreqOffense     = OffenseDF.orderBy(col("Total").desc()).first()
leastFreqLawCategory      = LawCategoryDF.orderBy(col("Total").asc()).first()
mostFreqLawCategory       = LawCategoryDF.orderBy(col("Total").desc()).first()
leastFreqBorough     = BoroughDF.orderBy(col("Total").asc()).first()
mostFreqBorough     = BoroughDF.orderBy(col("Total").desc()).first()
leastFreqPremises         = PremisesDF.orderBy(col("Total").asc()).first()
mostFreqPremises          = PremisesDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqOffense", "mostFreqOffense", "leastFreqLawCategory", "mostFreqLawCategory", \
       "%s (%d occurrences)" % (leastFreqOffense["OFNS_DESC"], leastFreqOffense["Total"]), \
       "%s (%d occurrences)" % (mostFreqOffense["OFNS_DESC"], mostFreqOffense["Total"]), \
       "%s (%d occurrences)" % (leastFreqLawCategory["LAW_CAT_CD"], leastFreqLawCategory["Total"]), \
       "%s (%d occurrences)" % (mostFreqLawCategory ["LAW_CAT_CD"], mostFreqLawCategory["Total"]))))

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqBorough", "mostFreqBorough", "leastFreqPremises", "mostFreqPremises", \
       "%s (%d occurrences)" % (leastFreqBorough["BORO_NM"], leastFreqBorough["Total"]), \
       "%s (%d occurrences)" % (mostFreqBorough["BORO_NM"], mostFreqBorough["Total"]), \
       "%s (%d occurrences)" % (leastFreqPremises["PREM_TYP_DESC"], leastFreqPremises["Total"]), \
       "%s (%d occurrences)" % (mostFreqPremises["PREM_TYP_DESC"], mostFreqPremises["Total"]))))

Summary of columns Offense, LawCategory, Borough, PremisesDesc:
+-------+--------------------+----------+-------------+------------------+
|summary|           OFNS_DESC|LAW_CAT_CD|      BORO_NM|     PREM_TYP_DESC|
+-------+--------------------+----------+-------------+------------------+
|  count|              467403|    468576|       468576|            466516|
|   mean|                null|      null|         null|              null|
| stddev|                null|      null|         null|              null|
|    min|            ABORTION|    FELONY|        BRONX|ABANDONED BUILDING|
|    25%|                null|      null|         null|              null|
|    50%|                null|      null|         null|              null|
|    75%|                null|      null|         null|              null|
|    max|VEHICLE AND TRAFF...| VIOLATION|STATEN ISLAND|       VIDEO STORE|
+-------+--------------------+----------+-------------+------------------+

Checking for nulls on columns Offen


| leastFreqOffense | mostFreqOffense | leastFreqLawCategory | mostFreqLawCategory |
|----|----|----|----|
| OTHER TRAFFIC INFRACTION (1 occurrences) | PETIT LARCENY (80205 occurrences) | VIOLATION (62306 occurrences) | MISDEMEANOR (260133 occurrences) |



| leastFreqBorough | mostFreqBorough | leastFreqPremises | mostFreqPremises |
|----|----|----|----|
| STATEN ISLAND (21714 occurrences) | BROOKLYN (140351 occurrences) | TRAMWAY (6 occurrences) | STREET (143398 occurrences) |


### 5. Business Questions
A. Seasonality of Crimes 

In [18]:
from pyspark.sql.functions import count, round
totalCrimes = NYcrimesDF.count()


SeasonDF = NYcrimesDF\
.withColumn('Season', when((col("month") == 12) | (col("month")== 1)| (col("month")==2), "Winter")\
           .when((col("month")> 2) & (col("month")<= 5), "Spring")\
           .when((col("month")> 5) & (col("month")<= 8), "Summer")\
           .otherwise("Automn"))


SeasonDF.cache() 

SeasonDF.select("Season", "month")\
    .groupBy("Season")\
    .agg(count("Season").alias("totalCrimes"), \
                          (count("Season")/ totalCrimes *100).alias("%"))\
    .orderBy("Season")\
    .select("Season", "totalCrimes", round("%",2).alias("Ratio"))\
    .show()


+------+-----------+-----+
|Season|totalCrimes|Ratio|
+------+-----------+-----+
|Automn|     118907|25.38|
|Spring|     119311|25.46|
|Summer|     129358|27.61|
|Winter|     101000|21.55|
+------+-----------+-----+



### C. Top Crimes per Borough 

In [182]:
NYcrimesBoroughDF = \
   NYcrimesDF.groupBy("BORO_NM")\
             .agg(count("OFNS_DESC").alias("Total Crimes"))\
             .orderBy(col("Total Crimes").desc())


combinedDF = \
  mostfreqperboroughDF\
     .join(NYcrimesBoroughDF, "BORO_NM")\

combinedDF.cache()
display(Markdown(""))
combinedDF.limit(20).show()

+-------------+-------------+------------------+------------+
|      BORO_NM|MostFreqCrime|MostFreqOccurances|Total Crimes|
+-------------+-------------+------------------+------------+
|     BROOKLYN|PETIT LARCENY|             22331|      140085|
|    MANHATTAN|PETIT LARCENY|             24397|      110196|
|        BRONX|PETIT LARCENY|             14678|      102712|
|       QUEENS|PETIT LARCENY|             15525|       92732|
|STATEN ISLAND|HARRASSMENT 2|              4310|       21678|
+-------------+-------------+------------------+------------+



In [146]:
BrooklynDF = \
   NYcrimesDF.where((col("BORO_NM")== "BROOKLYN"))\
            .groupBy("OFNS_DESC", "BORO_NM")\
            .agg(count("OFNS_DESC").alias("Occurances"))\
            .sort(col("Occurances").desc())\
            .first()

BrooklynDF

Row(OFNS_DESC='PETIT LARCENY', BORO_NM='BROOKLYN', Occurances=22331)

In [145]:
ManhattanDF = \
   NYcrimesDF.where((col("BORO_NM")== "MANHATTAN"))\
            .groupBy("OFNS_DESC", "BORO_NM")\
            .agg(count("OFNS_DESC").alias("Occurances"))\
            .sort(col("Occurances").desc())\
            .first()

ManhattanDF

Row(OFNS_DESC='PETIT LARCENY', BORO_NM='MANHATTAN', Occurances=24397)

In [144]:
BronxDF = \
   NYcrimesDF.where((col("BORO_NM")== "BRONX"))\
            .groupBy("OFNS_DESC", "BORO_NM")\
            .agg(count("OFNS_DESC").alias("Occurances"))\
            .sort(col("Occurances").desc())\
            .first()

BronxDF

Row(OFNS_DESC='PETIT LARCENY', BORO_NM='BRONX', Occurances=14678)

In [143]:
QueensDF = \
   NYcrimesDF.where((col("BORO_NM")== "QUEENS"))\
            .groupBy("OFNS_DESC", "BORO_NM")\
            .agg(count("OFNS_DESC").alias("Occurances"))\
            .sort(col("Occurances").desc())\
            .first()

QueensDF

Row(OFNS_DESC='PETIT LARCENY', BORO_NM='QUEENS', Occurances=15525)

In [161]:
statenislandDF = \
   NYcrimesDF.where((col("BORO_NM")== "STATEN ISLAND"))\
            .groupBy("OFNS_DESC", "BORO_NM")\
            .agg(count("OFNS_DESC").alias("Occurances"))\
            .sort(col("Occurances").desc())\
            .first()

statenislandDF

Row(OFNS_DESC='HARRASSMENT 2', BORO_NM='STATEN ISLAND', Occurances=4310)

In [192]:
from pyspark.sql.types import *
from pyspark.sql import Row

schema = StructType([StructField('BORO_NM', StringType()), StructField('MostFreqCrime',StringType()), StructField('MostFreqOccurances',IntegerType())])
rows = [ 
        Row(BORO_NM ='BROOKLYN', MostFreqCrime = 'PETIT LARCENY', MostFreqOccurances = 22331),
        Row(BORO_NM ='MANHATTAN', MostFreqCrime = 'PETIT LARCENY', MostFreqOccurances = 24397),
        Row(BORO_NM ='BRONX', MostFreqCrime = 'PETIT LARCENY', MostFreqOccurances = 14678),
        Row(BORO_NM ='QUEENS', MostFreqCrime = 'PETIT LARCENY', MostFreqOccurances = 15525),
        Row(BORO_NM ='STATEN ISLAND', MostFreqCrime = 'HARRASSMENT 2', MostFreqOccurances = 4310)
       ]

mostfreqperboroughDF = spark.createDataFrame(rows, schema)


mostfreqperboroughDF.show()




+-------------+-------------+------------------+
|      BORO_NM|MostFreqCrime|MostFreqOccurances|
+-------------+-------------+------------------+
|     BROOKLYN|PETIT LARCENY|             22331|
|    MANHATTAN|PETIT LARCENY|             24397|
|        BRONX|PETIT LARCENY|             14678|
|       QUEENS|PETIT LARCENY|             15525|
|STATEN ISLAND|HARRASSMENT 2|              4310|
+-------------+-------------+------------------+

